In [1]:
# Libraries

from scrapping import get_data
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
# Functions

def count_skips(df):
    counts = {key: 0 for key in numbers}
    for columns in df:
        contador = 0
        for i in reversed(df[columns]):
            if i is not True:
                contador += 1
            else:
                counts[columns] = contador
                break
    return counts

In [3]:
# Load the data base and obtain the first DataFrame
db = pd.read_parquet('db.parquet')
winners = db.iloc[:, 2:7]

# Search in every row and convert it to another DataFrame with boolean results, where True means that the number won, and False means that it loose
for e in range(1, 6):
    for i in range(1, 51):
        winners.loc[winners[f"Nro{e}"] == i, f"{i}"] = True

winners.drop(columns = ["Nro1","Nro2","Nro3","Nro4","Nro5"], inplace = True)

    # Add an extra row, in order to compare the number that did not appear for the first time in the game history
    
d_0 = pd.DataFrame(columns = [str(i) for i in range (1, 51)], index = [0]).fillna(True)

# Create the final DataFrame
winners = pd.concat([d_0, winners]).reset_index(drop = True)

In [4]:
draws = list(np.arange(1, len(winners)))
numbers = [str(i) for i in range(1, 51)]
dicts = {draw: {key:[] for key in numbers} for draw in draws}

for e in draws:
    df = winners.loc[:e]
    counts = count_skips(df)
    dicts[e].update(counts)

skip_numbers = pd.DataFrame(dicts).T

In [5]:
# Order the last draw for skips:
last_draw = sorted(dicts[len(skip_numbers)].items(), key = lambda x: x[1])
last_draw = pd.DataFrame({'Numero': [x[0] for x in last_draw], 'Skips': [int(x[1]) for x in last_draw]})

# Select  the last 12 draws
last_12_draws = np.arange(len(skip_numbers) - 12, len(skip_numbers) + 1)
sk_12 = skip_numbers.loc[last_12_draws]

# Count the numbers that hasn´t appeared in the lasts draws
skips = np.arange(0, 19)
aus = []
last_7 = []
last_12 = []

for i in last_12_draws[1:13]:
    for column in sk_12:
        if sk_12.loc[i, str(column)] == 0:
            aus.append(sk_12.loc[i - 1, str(column)])

for number in skips:
    last_12.append(aus.count(number))
    last_7.append(aus[25:60].count(number))

# Create a DataFrame of the counts for skips in each of the last 7 and 12 draws
skips_7_12 = pd.DataFrame(np.column_stack([last_7, last_12]), columns = [7, 12])

In [6]:
groups = [list(range(i, i + 10)) for i in range(1, 51, 10)]
group_names = [tuple(range(i, i + 10)) for i in range(1, 51, 10)]
groups_of_winners = db.iloc[:, 2:7]
results = {i: {group_name: sum([1 for num in row if num in group]) for group_name, group in zip(group_names, groups)} for i, row in groups_of_winners.iterrows()}
results_df = pd.DataFrame.from_dict(results, orient = 'index')
results_df.columns = ['{}_to_{}'.format(i, i + 9) for i in range(1, 51, 10)]

In [7]:
db

,Dates,Sorteos,Nro1,Nro2,Nro3,Nro4,Nro5,Star_1,Star_2
0,2004-02-13,1,16,29,32,36,41,7,9
1,2004-02-20,2,7,13,39,47,50,2,5
2,2004-02-27,3,14,18,19,31,37,4,5
3,2004-03-05,4,4,7,33,37,39,1,5
4,2004-03-12,5,15,24,28,44,47,4,5
...,...,...,...,...,...,...,...,...,...
1604,2023-02-07,1605,2,8,34,44,47,3,9
1605,2023-02-10,1606,3,13,25,36,47,3,8
1606,2023-02-14,1607,24,26,38,43,46,2,3
1607,2023-02-17,1608,8,23,27,42,45,1,9


In [8]:
skip_numbers

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
1,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,0,2,2,2,...,1,2,2,2,2,2,0,2,2,0
3,3,3,3,3,3,3,1,3,3,3,...,2,3,3,3,3,3,1,3,3,1
4,4,4,4,0,4,4,0,4,4,4,...,3,4,4,4,4,4,2,4,4,2
5,5,5,5,1,5,5,1,5,5,5,...,4,5,5,0,5,5,0,5,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,14,0,5,7,26,5,2,0,3,31,...,4,5,2,0,1,6,0,11,41,6
1606,15,1,0,8,27,6,3,1,4,32,...,5,6,3,1,2,7,0,12,42,7
1607,16,2,1,9,28,7,4,2,5,33,...,6,7,0,2,3,0,1,13,43,8
1608,17,3,2,10,29,8,5,0,6,34,...,7,0,1,3,0,1,2,14,44,9


In [9]:
last_draw

,Numero,Skips
0,21,0
1,22,0
2,34,0
3,41,0
4,49,0
5,8,1
6,23,1
7,27,1
8,42,1
9,45,1


In [10]:
skips_7_12

,7,12
0,3,4
1,1,2
2,2,4
3,4,7
4,1,2
5,2,3
6,1,2
7,6,7
8,1,2
9,2,4


In [11]:
results_df

,1_to_10,11_to_20,21_to_30,31_to_40,41_to_50
0,0,1,1,2,1
1,1,1,0,1,2
2,0,3,0,2,0
3,2,0,0,3,0
4,0,1,2,0,2
...,...,...,...,...,...
1604,2,0,0,1,2
1605,1,1,1,1,1
1606,0,0,2,1,2
1607,1,0,2,0,2


In [12]:
# values = list(set(val for dic in [ultimo_sorteo] for val in dic.values()))